<a href="https://colab.research.google.com/github/buildist/ESRGANColab/blob/master/ESRGANColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 1. Initial setup (run once)

!git clone https://github.com/xinntao/ESRGAN.git
%cd ESRGAN
!git checkout tags/old-arch

In [0]:
# 2. Add models (run once per model file)
# Must be direct download URL pointing to a .pth file.

model_url = "https://www.dropbox.com/s/vouc15j8jjp2o5n/RRDB_ESRGAN_x4_old_arch.pth?dl=0"
!wget $model_url --content-disposition -P models

In [0]:
# 3a. Upload images

%cd LR
from google.colab import files
files.upload()
%cd ..

In [0]:
# 3b. Or add images by URL

image_url = ""
!wget $image_url --content-disposition -P LR

In [0]:
#@title 4. Upscale!

model_name = 'RRDB_ESRGAN_x4_old_arch.pth' #@param {type:"string"}
upscale = 4 #@param {type:"number"}

import sys
import os.path
import glob
import cv2
import numpy as np
import torch
import architecture as arch
from IPython import display as ipythondisplay
from IPython.display import Image as ipythonimage

device = torch.device('cuda')
model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=upscale, norm_type=None, act_type='leakyrelu', \
                        mode='CNA', res_scale=1, upsample_mode='upconv')
model.load_state_dict(torch.load('models/{:s}'.format(model_name)), strict=True)
model.eval()
for k, v in model.named_parameters():
    v.requires_grad = False
model = model.to(device)

for path in glob.glob('LR/*'):
    base = os.path.splitext(os.path.basename(path))[0]
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    path = 'results/{:s}_rlt.png'.format(base)
    cv2.imwrite(path, output)
    ipythondisplay.display(ipythonimage(str(path)))

In [0]:
# 5. Remove LR images (run when needed)

!rm LR/*.*